In [ ]:
import numpy as np

# Image and parameters
img = np.array([
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 2, 2, 2],
    [2, 2, 3, 3]
])
levels = 4  # Gray levels (0-3)

# Co-occurrence matrix calculator
def glcm(image, dx, dy, levels):
    matrix = np.zeros((levels, levels), dtype=int)
    rows, cols = image.shape
    for i in range(rows):
        for j in range(cols):
            ni, nj = i + dx, j + dy
            if 0 <= ni < rows and 0 <= nj < cols:
                matrix[image[i, j], image[ni, nj]] += 1
    return matrix

# Compute GLCMs for all directions
directions = {
    "P45° (d=1)": (-1, 1),
    "P90° (d=1)": (-1, 0),
    "P135° (d=1)": (-1, -1),
    "P0° (d=2)": (0, 2)
}
glcms = {name: glcm(img, dx, dy, levels) for name, (dx, dy) in directions.items()}

# Compute features for P0° (d=1)
P = glcm(img, 0, 1, levels) / np.sum(glcm(img, 0, 1, levels))  # Normalize
i, j = np.indices(P.shape)

# Haralick features
features = {
    "Energy": np.sum(P**2),
    "Entropy": -np.sum(P[P > 0] * np.log2(P[P > 0])),
    "Max Probability": np.max(P),
    "Contrast": np.sum((i - j)**2 * P),
    "IDM": np.sum(P / (1 + (i - j)**2)),
    "Correlation": (np.sum((i - np.sum(P * i)) * (j - np.sum(P * j)) * P) / 
                   (np.std(i) * np.std(j)))
}

print("GLCMs:")
for name, matrix in glcms.items():
    print(f"{name}:\n{matrix}\n")

print("Features for P0° (d=1):")
for name, value in features.items():
    print(f"{name}: {value:.4f}")

In [ ]:
import numpy as np

img = np.array([
    [0, 0, 1, 1],
    [0, 0, 1, 1],
    [0, 2, 2, 2],
    [2, 2, 3, 3]
])

def glcm_counts(image, levels, dx, dy):
    glcm = np.zeros((levels, levels), dtype=int)
    rows, cols = image.shape
    for i in range(rows):
        for j in range(cols):
            ni, nj = i + dx, j + dy
            if 0 <= ni < rows and 0 <= nj < cols:
                a = image[i, j]
                b = image[ni, nj]
                glcm[a, b] += 1
    return glcm

# Number of gray levels (0 through 3)
levels = 4

# Offsets for requested directions/distances
offsets = {
    "P45° (d=1)": (-1, 1),
    "P90° (d=1)": (-1, 0),
    "P135° (d=1)": (-1, -1),
    "P0° (d=2)": (0, 2)
}

# Compute and display co-occurrence matrices
glcms = {}
for name, (dx, dy) in offsets.items():
    glcm = glcm_counts(img, levels, dx, dy)
    glcms[name] = glcm


# Compute features for P0° (d=1) -> offset (0, 1)
glcm01 = glcm_counts(img, levels, 0, 1)
P = glcm01 / glcm01.sum()  # convert to probabilities

# Compute Haralick features
# Energy
energy = np.sum(P**2)

# Entropy (log base 10)
entropy = -np.sum(P[P > 0] * np.log2(P[P > 0]))

# Maximum probability
max_prob = np.max(P)

# Contrast
i, j = np.indices(P.shape)
contrast = np.sum((i - j)**2 * P)

# Inverse Difference Moment (IDM)
idm = np.sum(P / (1 + (i - j)**2))

# Correlation
px = np.sum(P, axis=1)
py = np.sum(P, axis=0)
levels_range = np.arange(levels)
ux = np.sum(px * levels_range)
uy = np.sum(py * levels_range)
sigx = np.sqrt(np.sum(px * (levels_range - ux)**2))
sigy = np.sqrt(np.sum(py * (levels_range - uy)**2))
correlation = np.sum((i - ux) * (j - uy) * P) / (sigx * sigy)

print("Features for P0° (d=1):")
print(f"Energy: {energy}")
print(f"Entropy (log2): {entropy}")
print(f"Maximum Probability: {max_prob}")
print(f"Contrast: {contrast}")
print(f"Inverse Difference Moment: {idm}")
print(f"Correlation: {correlation}")


In [ ]:
import numpy as np

# Training data
omega1 = [(4,4),(14,7),(18,6),(7,4),(14,9),(16,2),(2,16)]
omega2 = [(-7,8),(3,11),(-2,-1),(4,3),(3,4),(4,-3),(0,6)]
omega3 = [(3,2),(8,1),(3,3),(6,3),(5,10),(3,9),(1,7)]

test_point = (4,3)

def euclidean_distance(a, b):
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

# Calculate all distances
distances = []
for i, points in enumerate([omega1, omega2, omega3]):
    for point in points:
        dist = euclidean_distance(test_point, point)
        class_label = i+1  # ω1=1, ω2=2, ω3=3
        distances.append((dist, point, class_label))  # Store distance, point, and class

# Sort distances
distances.sort()

# Print all distances (for debugging)
print("=== All distances sorted ===")
for dist, point, class_label in distances:
    print(f"Point {point} from ω{class_label}: Distance = {dist:.2f}")

# Function to classify based on K
def classify_k_nearest(k):
    k_nearest = distances[:k]
    votes = {}
    print(f"\n=== {k}-Nearest Neighbors ===")
    for dist, point, class_label in k_nearest:
        print(f"Point {point} from ω{class_label}: Distance = {dist:.2f}")
        votes[class_label] = votes.get(class_label, 0) + 1
    print(f"Votes: {votes}")
    return max(votes, key=votes.get)

# Display results
print("\n=== Classification Results ===")
print("a) 1-NN classification: ω" + str(classify_k_nearest(1)))
print("b) 3-NN classification: ω" + str(classify_k_nearest(3)))
print("c) 7-NN classification: ω" + str(classify_k_nearest(7)))